In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("assignment4.2.ipynb")

# DS 453 / 653: Programming Assignment 4

**Due date**: Thursday, Feburary 15 at 8pm on [Gradescope](https://www.gradescope.com/courses/710247).

_You must follow the Academic Code of Conduct and Collaboration Policy stated in the course syllabus at all times while working on this assignment._

This assignment contains 2 questions worth a total of 5 points. You must receive at least 4 points to pass the assignment.

To begin, please execute the code block below:

## Assignment Overview

The goal of this assignment is to continue our investigation into the structure of a blockchain. We'll continue to use the toy blockchain from Programming Assignment 3.

[Everything else in this assignment overview is identical to what we wrote in Assignment 3; it is here to remind you of how the toy blockchain works, and so that you execute the appropriate Python commands again.]

### Python classes
Our toy blockchain is written as Python code, which we provide below. We separate the blockchain into four classes:
- The `TransactionPayload` class holds the relevant information for a single payment: the sender and receiver (identified by their public keys) and the amount of money being transferred.
- The `Transaction` class contains the payload together with a digital signature made by the sender.
- The `Block` class contains the block "height" (which is just a 0-indexed counter), a set of new transactions, and a pointer to the previous block in the chain. Additionally, the block contains a `nonce` that can be chosen arbitrarily by the block's miner in order to satisfy the difficulty rule.

Finally, the `blockchain` object is a list of blocks.

Throughout this assignment, the difficulty rule is that:
> _The hash of a valid block must begin with 8 bits (one byte) of all 0s_.

To instantiate these classes, execute the Python code below.

In [2]:
import otter
grader = otter.Notebook()

In [3]:
# Execute this block only if you are using Google Colab.
# If you are running the notebook file locally, install pycryptodome yourself but do NOT install the package pycrypto.

# %pip install pycryptodome

In [4]:
## Execute, but DO NOT MODIFY this code block. ##
## It contains a Python class corresponding to a toy blockchain.
## Please read the code to understand how the toy blockchain works.

import json
from Crypto.Hash import SHA256
from Crypto.PublicKey import ECC
from Crypto.Signature import DSS
from binascii import hexlify, unhexlify

class Transaction():
  def __init__(self, transaction_payload, signature):
    self.transaction_payload = transaction_payload
    self.signature = signature

  def json_encode(self):
    transaction_json = {"transaction": json.loads(self.transaction_payload.json_encode()), "signature":self.signature}
    return json.dumps(transaction_json)

class TransactionPayload():
  def __init__(self, sender_public_key, receiver_public_key, amount):
    self.sender_public_key = sender_public_key
    self.receiver_public_key = receiver_public_key
    self.amount = amount

  def json_encode(self):
    return json.dumps(self.__dict__, sort_keys=True)

  def encode(self):
    # encodes the transaction as bytes
    return self.json_encode().encode('ascii')

  def hash(self):
    # returns the hash of the transaction as bytes
    return SHA256.new(self.encode())

class Block():
  def __init__(self, height, transactions, previous_hash, nonce=0):
    self.height = height
    self.transactions = transactions
    self.previous_hash = previous_hash
    self.nonce = nonce

  def json_encode(self):
    json_encoding = {'height': self.height,
            'transactions': [json.loads(transaction.json_encode()) for transaction in self.transactions],
            'previous_hash': self.previous_hash, # in hexstring
            'nonce': self.nonce
    }
    return json.dumps(json_encoding)

  def encode(self):
    return self.json_encode().encode('ascii')

  def hash(self):
    return SHA256.new(self.encode()).hexdigest()


### Participants in the blockchain

There are five participants who use this blockchain. We label these participants as players A, B, C, D, and E. We will only look at a few blocks of this blockchain, which operate as follows:

- Before the start of the blockchain, we assume that players A and C start with 1 coin. The other players have no money at the start.
- The first block contains two transactions: A pays 1 coin to B, and C pays 1 coin to D.
- The second block contains a single transaction: D pays 1 coin to A

After these two blocks, players A and B should have 1 coin each and everyone else should have nothing.

Execute the Python code below to instantiate this blockchain.

In [5]:
## Execute, but DO NOT MODIFY this code block. ##
## This code creates a concrete blockchain containing the transactions and blocks described above.
## It also contains all 5 players' public keys, and the corresponding secret key *only* for Player A.

# Player A's secret key
secret_key = ECC.generate(curve='P-256', randfunc=(lambda x: '0'.encode('ascii') * x))
# Public keys for all five players
# Player A's public key is located at index 0, player B's public key is at index 1, and so on
public_keys = ['044d4385fe08e0eb94c524bdd3682c9c5ae358f52402df02418d0ba6cf6889289a27bd56a666df7dc595c98da1f22958009ae17c52fd290185d2e2bf11140a0a5e', '04be4a2555cc8ed29989eeddde3d8e4d41458d02dce047aac2e6af2d58688c5beae7163eb157d1fd3c3d282abab7172b0f0d63274ebe721d31c5d72a83741df170', '04a3fc575afc4ad2ca2cbeed50b52fedf049d317b790aa33e7f8182aec028453082d4c4ffd4c99c7e6d11c134e81020c57ca0fbf0bf3406bf4457dc72d8c994a65', '04abc5bd2786ea5144122ccf43b89a557e1d36e2773c2b8986d9819a22c2e6540b3d5da692504ccc29fbf774435312afdda2a360d5160329cb326f6d47db29f50b', '046a921bed68e07e66f38a5137b3784372cfd4507e6451e1dfe3760f08649750e7891cd30339b730c0280738c60233d3c99eb78454340ff363cafbf1e6f0cfce5e']

blockchain = []

# Remember: we assume that A and C magically begin the blockchain with 1 coin.
# This is because our toy blockchain doesn't have mining rewards,
# so we make this assumption to start the chain with some money.

# Here are the transactions in each block
def gen_block_0():
  # This block sends 1 coin from public key A to public key B
  t1 = TransactionPayload(public_keys[0], public_keys[1], 1)
  # In this toy blockchain, all signatures are stored in hex format and have a type of string
  t1_signature = "edd6bb60e8486f9747f911a6d7340c275d5a417c3823952715a18a64d3577325413cf544e8f48e6c30a312649200c57b72d7eaa4721abf376dbd0dc799db6c3a"
  # A transaction includes the payload and its corresponding signature
  t1_signed = Transaction(t1, t1_signature)

  # C -> D
  t2 = TransactionPayload(public_keys[2], public_keys[3], 1)
  t2_signature = "33bcaaa920207125ec72ea7f9e5fbddb993479dbe7122c6dfcf0a96f2e646dd90ff73c90c784f473480e32fd2fb68735af86458bc0434e5958f025cc9e0ddd59"
  t2_signed = Transaction(t2, t2_signature)

  # block number 0
  previous_hash = "0000000000000000000000000000000000000000000000000000000000000000"
  b0 = Block(0, [t1_signed, t2_signed], previous_hash, 7)
  return b0

def gen_block_1():
  # D -> A
  t3 = TransactionPayload(public_keys[3], public_keys[0], 1)
  t3_signature = "886de552f43010b2a848f660b066102e680a4c6b763032c98c53e6f86c5d5638a9dee5e41f311e4956cd416a51850afa28bc4b9575cbf9cbf1940fae84b1d166"
  t3_signed = Transaction(t3, t3_signature)

  # block number 1
  previous_hash = "00fd71161f8b5244d0641d83ccaedc2f0b7bed9d3f0c625fc817218660c4369a"
  b1 = Block(1, [t3_signed], previous_hash, 143)
  return b1

def pretty_print(blockchain):
  [print(block.json_encode()) for block in blockchain]

blockchain = [gen_block_0(), gen_block_1()]
pretty_print(blockchain)


{"height": 0, "transactions": [{"transaction": {"amount": 1, "receiver_public_key": "04be4a2555cc8ed29989eeddde3d8e4d41458d02dce047aac2e6af2d58688c5beae7163eb157d1fd3c3d282abab7172b0f0d63274ebe721d31c5d72a83741df170", "sender_public_key": "044d4385fe08e0eb94c524bdd3682c9c5ae358f52402df02418d0ba6cf6889289a27bd56a666df7dc595c98da1f22958009ae17c52fd290185d2e2bf11140a0a5e"}, "signature": "edd6bb60e8486f9747f911a6d7340c275d5a417c3823952715a18a64d3577325413cf544e8f48e6c30a312649200c57b72d7eaa4721abf376dbd0dc799db6c3a"}, {"transaction": {"amount": 1, "receiver_public_key": "04abc5bd2786ea5144122ccf43b89a557e1d36e2773c2b8986d9819a22c2e6540b3d5da692504ccc29fbf774435312afdda2a360d5160329cb326f6d47db29f50b", "sender_public_key": "04a3fc575afc4ad2ca2cbeed50b52fedf049d317b790aa33e7f8182aec028453082d4c4ffd4c99c7e6d11c134e81020c57ca0fbf0bf3406bf4457dc72d8c994a65"}, "signature": "33bcaaa920207125ec72ea7f9e5fbddb993479dbe7122c6dfcf0a96f2e646dd90ff73c90c784f473480e32fd2fb68735af86458bc0434e5958f025cc9e0

This `blockchain` turns out to be valid because:
1. All signatures are correct,
2. Everyone has enough money in their account at the moment they post a transaction, 
3. Every block points to the previous block (and by convention, the initial block -- called the "genesis" block -- has a hash of all-0s), and
4. Every block has a `nonce` that is chosen to satisfy the difficulty rule.

If we break any of the four rules, then the blockchain becomes invalid. Here are some examples of invalid blockchains.

In [6]:
# # An invalid chain because a digital signature is incorrect
# # (specifically, we're altering the signature of the first A -> B transaction)
# chain_invalid_sig = blockchain
# chain_invalid_sig[0].transactions[0].signature = "00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000"

# # An invalid chain because someone is spending money they don't have
# # (specifically, we're moving the D -> A transaction into the first block)
# chain_invalid_balance = blockchain
# chain_invalid_sig[0].transactions[0].transaction_payload = TransactionPayload(public_keys[3], public_keys[0], 1)
# chain_invalid_sig[0].transactions[0].signature = "886de552f43010b2a848f660b066102e680a4c6b763032c98c53e6f86c5d5638a9dee5e41f311e4956cd416a51850afa28bc4b9575cbf9cbf1940fae84b1d166"

# # An invalid chain with the wrong hash of the previous block
# chain_invalid_hash = blockchain
# chain_invalid_hash[1].previous_hash = '2222222222222222222222222222222222222222222222222222222222222222'

# # An invalid chain that doesn't satisfy the difficulty rule
# chain_invalid_difficulty = blockchain
# chain_invalid_difficulty[1].nonce = 142

## Helper Functions

In [7]:
from typing import Union

def isValidTransaction(transaction: Transaction) -> bool: # GOOD 
    # Todo: write here!
    # return type: bool

    payload = transaction.transaction_payload

    key = ECC.import_key(unhexlify(payload.sender_public_key), curve_name='P-256')
    msg = payload.hash()
    verifier = DSS.new(key, 'fips-186-3')

    try:
        verifier.verify(msg, unhexlify(transaction.signature))
        return True
    except ValueError:
        return False

def verifyBlock(block: Block) -> bool: # GOOD
    # Todo: write here!
    # return type: bool

    # diffulty check
    block_hash = block.hash()
    if block_hash[:2] != "00":
        return False

    # transaction check
    transactions = block.transactions
    for transaction in transactions:
        if not isValidTransaction(transaction):
            return False
        
    return True

# class Block():
#   def __init__(self, height, transactions, previous_hash, nonce=0):
#     self.height = height
#     self.transactions = transactions
#     self.previous_hash = previous_hash
#     self.nonce = nonce

def findBalance(blockchain: list[Block], players_balance: list[int]) -> Union[list[int], bool]:
    # input: a blockchain and an integer balance
    # output: either the new list of int, or the bool `False`
    
    for block in blockchain:

        # check if block has the correct previous hash
        if block.height != 0:
            if block.previous_hash != blockchain[block.height - 1].hash():
                return False
        else:
            # check if first block is valid
            if block.previous_hash != "0000000000000000000000000000000000000000000000000000000000000000":
                return False

        # check if block is valid
        if not verifyBlock(block):
            return False

        # update balance
        for transaction in block.transactions:
            sender_index = public_keys.index(transaction.transaction_payload.sender_public_key)
            receiver_index = public_keys.index(transaction.transaction_payload.receiver_public_key)
            amount = transaction.transaction_payload.amount

            if players_balance[sender_index] < amount:
                return False

            players_balance[sender_index] -= amount
            players_balance[receiver_index] += amount
    
    return players_balance

## Assignment Questions

In this week's assignment, your objective is to take on the role of a *miner* and to create new blocks in the toy blockchain. First, you will act as an honest miner and produce a new block legitimately, i.e., extending the end of the `blockchain` that you already have. Second, you will act as a malicious miner and produce a new block illegitimately, by overwriting the prior state on the blockchain. The difficulty level of the toy blockchain is purposely tuned down so that this is possible to do.

**Question 1 (Honest mining):** Mine a third block as player A. Your new block should include a single transaction: as player A, transfer your one and only coin to player E.

You may use the functions `isValidTransaction`, `verifyBlock`, and `findBalance` from Programming Assignment 3 as helper functions within this code, if you wish.

To start, please write the method `computeSignature` to generate the signature, as player A, corresponding to a transaction payload. This function should return a signature in hexadecimal string format. Remember that you will need to use player A's secret key to produce a signature.

In this assignment, you must use a digital signature scheme called ECDSA, which stands for Elliptic Curve Digital Signature Algorithm. We discussed this algorithm in Lecture 4. You should review the PyCrypto documentation at https://pycryptodome.readthedocs.io/en/latest/src/signature/dsa.html?highlight=ecdsa to see a working example.

The following is the code for the test case:

```python
def q1_test():
    appended_blockchain = mineThirdBlock(blockchain)
    
    # 1. verify validity of blockchain
    players_balance = [1,0,1,0,0]
    for block in appended_blockchain:
        # 1. validate the blockchain
        if not verifyBlock(block):
            return False
    
    # 2. verify the final balance of each player is correct
    players_balance = findBalance(appended_blockchain, players_balance)
    if players_balance != [0, 1, 0, 0, 1]:
        return False
        
    # 3. check A -> E (1)
    new_block = appended_blockchain[-1]
    transactions = new_block.transactions
    if len(transactions) == 1:
        transaction = transactions[0]
        payload = transaction.transaction_payload
        if payload.sender_public_key == public_keys[0] and payload.receiver_public_key == public_keys[4] and payload.amount ==1 :
            return players_balance
    return False
```

_Points:_ 2

In [8]:
def computeSignature(transaction_payload: TransactionPayload): # GOOD
    # Todo: write here!
    # return type: hexadecimal string corresponding to a signature
    
    message = transaction_payload.encode()
    h = SHA256.new(message)
    signer = DSS.new(secret_key, 'fips-186-3')
    signature = signer.sign(h)

    return hexlify(signature).decode('ascii')

# test with isValidTransaction

In [9]:
isValidTransaction(blockchain[0].transactions[0])

True

In [10]:
# Next, write the method `mineBlock` to create a new block when given any set of transactions. 
# This function should return a valid Block object.
def mineBlock(blockchain: list[Block], transactions): # GOOD
    # Todo: write here!
    # return type: a Block object

    if len(blockchain) == 0:
        previous_hash = "0000000000000000000000000000000000000000000000000000000000000000"
    else:
        previous_hash = blockchain[-1].hash()

    # create a new block
    block = Block(
        len(blockchain), 
        transactions, 
        previous_hash, 
        0)
    
    # mine the block
    max = 10000
    while not verifyBlock(block):
        block.nonce += 1
        if block.nonce > max:
            return False
    #     print(block.nonce, end='\r')

    # print(block.nonce, block.hash()[:2])
    return block

# test with findBalance()

In [11]:
mineBlock(blockchain, [blockchain[0].transactions[0]])

In [12]:
# TODO: Finally, you should write a function called `mineThirdBlock` to first generate a new valid 
# block with required transaction using the `computeSignature` and `mineBlock` functions you wrote above, 
# and append it to the existing two-block `blockchain`. 
# The function should take the original blockchain as input and return the appended blockchain.

def mineThirdBlock(blockchain: list[Block]) -> list[Block]:
    
    payload = TransactionPayload(public_keys[0], public_keys[4], 1)

    new_transaction = Transaction(
        payload,
        computeSignature(payload)
    )

    new_block = mineBlock(blockchain, [new_transaction])
    blockchain.append(new_block)

    return blockchain

## Test Function

In [13]:
def q1_test():
    appended_blockchain = mineThirdBlock(blockchain)
    
    # 1. verify validity of blockchain
    players_balance = [1,0,1,0,0]
    for block in appended_blockchain:
        # 1. validate the blockchain
        if not verifyBlock(block):
            return False
    
    # 2. verify the final balance of each player is correct
    players_balance = findBalance(appended_blockchain, players_balance)
    if players_balance != [0, 1, 0, 0, 1]:
        return False
        
    # 3. check A -> E (1)
    new_block = appended_blockchain[-1]
    transactions = new_block.transactions
    if len(transactions) == 1:
        transaction = transactions[0]
        payload = transaction.transaction_payload
        if payload.sender_public_key == public_keys[0] and payload.receiver_public_key == public_keys[4] and payload.amount ==1 :
            return players_balance
    return False

In [14]:
grader.check("q1")

q1 results: All test cases passed!

**Question 2 (Dishonest mining):** Your objective in this task is to do something malicious: execute a double-spend attack! As player A, you should generate a blockchain that includes a transaction in which you pay **two coins** to player E.

But player A doesn't have 2 coins! So in order to make this attack work, you need to rewrite the state of the blockchain so that player A never pays 1 coin to player B (but still has 1 coin from the start and receives 1 coin from player D, thereby resulting in two coins that can be paid to player E).

Below, you must write code that will produce a new blockchain. Just like before, a blockchain is a list of valid blocks. To satisfy the requirements of this question, your new blockchain must be valid, and it must convince the nodes to switch from the current `blockchain` object to your new, malicious blockchain. Remember that Bitcoin nodes will always prefer the blockchain that has the most work.

You should create a method call `generate_dishonest_blockchain()` that returns `bad_blockchain`

The following is the test code:

```python
def q2_test():
    bad_blockchain: list[Block] = generate_dishonest_blockchain()

    # 1. check the length of the bad_blockchain 
    if len(bad_blockchain) < 3:
        return False

    # 2. test validity of the blockchain
    isValid = True
    for block in bad_blockchain:
        if not verifyBlock(block):
            isValid = False
        if isValid:
            return True
        else:
            return False
    
    # 3. check in Block 0, whether A never pay B

    # 4. check whether A pay E two coins

    return True
```

_Points:_ 3

In [15]:
# use the variable `bad_blockchain` to store your blockchain
bad_blockchain = []

def generate_dishonest_blockchain():
    # create block with all transactions except the first one
    remaining_transactions = blockchain[0].transactions[1:]
    block = mineBlock([], remaining_transactions)
    
    # append the new block and the first block to the bad blockchain
    bad_blockchain = [block] + blockchain[1:]

    # create a double spend transaction
    double_payload = TransactionPayload(public_keys[0], public_keys[4], 2)

    double_transaction = Transaction(
        double_payload,
        computeSignature(double_payload)
    )

    new_block = mineBlock(bad_blockchain, [double_transaction])
    bad_blockchain.append([new_block])

    return bad_blockchain


# Test Function 2

In [16]:
def q2_test():
    bad_blockchain: list[Block] = generate_dishonest_blockchain()

    # 1. check the length of the bad_blockchain 
    if len(bad_blockchain) < 3:
        return False

    # 2. test validity of the blockchain
    isValid = True
    for block in bad_blockchain:
        if not verifyBlock(block):
            isValid = False
        if isValid:
            return True
        else:
            return False
    
    # 3. check in Block 0, whether A never pay B

    # 4. check whether A pay E two coins

    return True

In [17]:
grader.check("q2")

q2 results: All test cases passed!

## Submitting the Assignment

Please follow these instructions to complete the assignment and submit it for credit.

**Documenting collaborators, sources, and AI tools:** In accordance with the collaboration policy, use the space below to report if you used any resources to complete this homework assignment, aside from the lecture notes and the course textbooks/videos. Specifically, please report:

1. Names of all classmates you worked with, and a short description of the work that you performed together.
2. All written materials that you used, such as books or websites (besides the lecture notes or textbooks). Please include links to any web-based resources, or citations to any physical works.
3. All code that you used from other sources. In particular, if you used an AI tool, then you must include the entire exchange with the AI tool, as per the [CDS Generative AI Assistance Policy](https://www.bu.edu/cds-faculty/culture-community/gaia-policy/).

Remember that if we discover any undocumented collaborators, sources, or AI tools then this is grounds for a grade penalty and referral to BU's Academic Conduct Committee (as described in the syllabus).

_Your response:_

1.

2.

3.

**Sending to Gradescope:** After completing the assignment:
- if you did the assignment on Colab, download it in `.ipynb` format.
- if you did the assignment locally on your machine, all you need to do is to find it in your directory.

Then, submit only the `.ipynb` file to this week's programming assignment on Gradescope. It may take a few seconds or a minute for the auto-grading system to check your work.

Remember that you can submit as many times as you want until the deadline for the assignment; only your last score counts.

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Submit your assignment on Gradescope once you have passed enough tests to receive at least 4 of the 5 available points.

In [18]:
# Save your notebook first, then run this cell to export your submission.
# grader.export(run_tests=True)